# Palmer Amaranth (Amaranthus palmeri) Growth Stage Detection
This Google Colab notebook has been developed from the official YOLOv6 ['Train on Custom Data' tutorial](https://github.com/meituan/YOLOv6/blob/main/docs/Train_custom_data.md). It can be used to train and evaluate YOLOv6 on the Palmer amaranth Growth Stage (PAGS8) dataset. The dataset is available for download from [Weed-AI](). Prior to use, the dataset should be setup using the instructions from the official repository.

Please note this uses the [2.0 release](https://github.com/meituan/YOLOv6/releases/tag/0.2.0) from the v6 repository.

It accompanies the preprint: **Multi-growth stage plant recognition: a case study of Palmer amaranth (Amaranthus palmeri) in cotton (Gossypium hirsutum)** available on [arXiv](https://arxiv.org/abs/2307.15816). Please consider citing this preprint if you use the work in your research.

Models were trained using Google Colab Pro+ with access to an NVIDIA A100-SXM4-40GB.

This notbook consists of three parts:
1. Setup
2. Training
3. Evaluation

(Remember to choose GPU in Runtime if not already selected. Runtime --> Change Runtime Type --> Hardware accelerator --> GPU)

<img src="https://github.com/geezacoleman/Palmer-detection/assets/51358498/6040227e-a072-43bf-a789-72e0833f3168"/>

# 1. Setup

In [ ]:
# mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
rootDir = '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/data'
%cd '/content/drive/MyDrive/Colab Notebooks/Palmer-detection/'

In [ ]:
%cd yolov6
!pip install -r requirements.txt

In [ ]:
# import necessary packages
import torch
import random

random.seed(0)

from datetime import datetime

DATE = datetime.now().strftime('%Y%m%d')
IMAGE_SIZE = 1280

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 1.13.1+cu116 _CudaDeviceProperties(name='NVIDIA A100-SXM4-40GB', major=8, minor=0, total_memory=40536MB, multi_processor_count=108)


## Download pretrained models
Download the pretrained models to finetune on the custom data into the 'weights' directory within YOLOv6. They should only be downloaded once.

NOTE: These are 0.2.0 models from the [2.0 release](https://github.com/meituan/YOLOv6/releases/tag/0.2.0). For the latest models please check the official repository.

In [ ]:
%cd ..
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6n.pt -O weights/yolov6n.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6s.pt -O weights/yolov6s.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6m.pt -O weights/yolov6m.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6t.pt -O weights/yolov6t.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6l.pt -O weights/yolov6l.pt
!wget https://github.com/meituan/YOLOv6/releases/download/0.2.0/yolov6l.pt -O weights/yolov6l_relu.pt

You can use Tensorboard to track training.

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# 2. Training

In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/Palmer-detection/yolov6'
models = ['n', 't', 's', 'm', 'l']
classes = ['8', '1']

# different folds
for model in models:
  for class_num in classes:
    for fold in range(0, 5):
      yaml_path = f"{class_num}cls_fold_{fold}.yaml"
      full_name = f'{DATE}_yolov6{model}_B8_F{fold}_{class_num}cls'
      
      print('################ STARTING {} ###################'.format(full_name))
      !python tools/train.py --workers 8 --batch 8 --device 0 --img-size {IMAGE_SIZE} --epochs 30 --conf configs/yolov6{model}_finetune.py  --data data/{data_path}  --name {full_name} --eval-final-only

# 3. Evaluation

In [ ]:
import os
train_path = "/content/drive/My Drive/Colab Notebooks/Palmer-detection/yolov6/runs/train"

for run_name in os.listdir(train_path):
  if 'v6' not in run_name:
    print(f'skipping {run_name}')
  
  else:
    fold = run_name.split("_")[-3][-1]
    class_num = run_name.split("_")[-2][:-3]
    model = run_name.split("_")[1][-1]
    
    yaml_path = f"{class_num}cls_fold_{fold}.yaml"
  
    !python tools/eval.py --device 0 --img-size {IMAGE_SIZE} --task 'test' --config-file configs/yolov6{model}_finetune_eval.py --weights runs/train/{run_name}/weights/best_ckpt.pt --data data/{data_name} --name {run_name}
